In [1]:
import asyncio

Let's consider the simplest possible coroutine for the sake of illustration

In [2]:
import time

async def mycoro(timeout):
    print("-> {} mycoro({})".format(time.strftime("%M-%S"), timeout))
    await asyncio.sleep(timeout)
    print("<- {} mycoro({})".format(time.strftime("%M-%S"), timeout))
    # return something easy to recognize
    return 1000 * timeout

# example 1

Running a series of coroutines in parallel - a la `gather` - can be done like this

In [3]:
from action import Action
from engine import Engine

In [4]:
a1, a2, a3 = Action(mycoro(1)), Action(mycoro(2)), Action(mycoro(1.5)),

What we're saying here is that we have three jobs, that have no relationships between them. 

So when we run them, we would start all 3 coroutines at once, and return once they are all done:

In [5]:
e = Engine(a1, a2, a3)
e.orchestrate()

-> 58-54 mycoro(1.5)
-> 58-54 mycoro(1)
-> 58-54 mycoro(2)
<- 58-55 mycoro(1)
<- 58-56 mycoro(1.5)
<- 58-56 mycoro(2)


True

# exemple 2 : add requirements (dependencies)

Now we can to add *requirements* dependencies between actions, like in the following example

In [6]:
b1, b2, b3 = Action(mycoro(1)), Action(mycoro(2)), Action(mycoro(1.5), label="last"),
b3.requires(b1)

Now `b3` needs `b1` to be finished before it can start. And so only the 2 first coroutines get started at the beginning, and only once b1 has finished does b3 start.

In [7]:
e2 = Engine(b1, b2, b3)
e2.orchestrate()

-> 58-57 mycoro(2)
-> 58-57 mycoro(1)
<- 58-58 mycoro(1)
-> 58-58 mycoro(1.5)
<- 58-59 mycoro(2)
<- 58-59 mycoro(1.5)


True

# Customize `Action`

`Action` actually is a specializtion of `AbstractAction`, and the specification is that the `corun()` method should denote a coroutine itself, and that's what is triggered by `Engine`.

# Inspect results

Details for each `Action` can be retrieved like this

In [8]:
print(b1.is_done())

True


In [9]:
print(b3.result())

1500.0


In [10]:
for action in e2.actions:
    print(action)

<Action `NO_LABEL' finished -> 2000>
<Action `last' finished -> 1500.0 - [requires [NO_LABEL]]>
<Action `NO_LABEL' finished -> 1000 - [allows [last]]>


# TODO

* this totally is only a seed at this point, like day d+1 
* robustify - what happens if `corun()` raises an exception ?
* come up with some basic (curses ?) monitor to show what's going on
